## The dataset - Kaggle.com/WalmartM5

# The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI). In this respect, the bottom-level of the hierarchy, i.e., product-store unit sales can be mapped across either product categories or geographical regions, as follows

# Table 1: Number of M5 series per aggregation level.
 Level 
id	Aggregation Level	Number of series
1	Unit sales of all products, aggregated for all stores/states	1
2	Unit sales of all products, aggregated for each State	3
3	Unit sales of all products, aggregated for each store 	10
4	Unit sales of all products, aggregated for each category	3
5	Unit sales of all products, aggregated for each department	7
6	Unit sales of all products, aggregated for each State and category	9
7	Unit sales of all products, aggregated for each State and department	21
8	Unit sales of all products, aggregated for each store and category	30
9	Unit sales of all products, aggregated for each store and department	70
10	Unit sales of product x, aggregated for all stores/states	3,049
11	Unit sales of product x, aggregated for each State	9,147
12	Unit sales of product x, aggregated for each store	30,490
Total	42,840 


In [3]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import regex as re
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder
import gc
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype


In [4]:
# These helper and data cleaning functions are from the fast.ai course
# The repository is here: https://github.com/fastai/fastai/tree/master

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
        
def make_date(df, date_field:str):
    "Make sure `df[field_name]` is of the right date type."
    field_dtype = df[date_field].dtype
    if isinstance(field_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        field_dtype = np.datetime64
    if not np.issubdtype(field_dtype, np.datetime64):
        df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
        

def add_datepart(df, fldnames, drop=True, time=False, errors="raise"):
    # add_datepart converts a column of df from a datetime64 to many columns containing the information from the date. 
    # This applies changes inplace.
    if isinstance(fldnames,str): 
        fldnames = [fldnames]
    for fldname in fldnames:
        fld = df[fldname]
        fld_dtype = fld.dtype
        if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
            fld_dtype = np.datetime64

        if not np.issubdtype(fld_dtype, np.datetime64):
            df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
        targ_pre = re.sub('[Dd]ate$', '', fldname)
        attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
                'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
        if time: attr = attr + ['Hour', 'Minute', 'Second']
        for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
        df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
        if drop: df.drop(fldname, axis=1, inplace=True)
        
        
def ifnone (a,b): #(a:Any,b:Any)->Any:
    "`a` if `a` is not None, otherwise `b`."
    return b if a is None else a

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


def train_cats(df):    
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

def apply_cats(df, trn):
    for n,c in df.items():
        if (n in trn.columns) and (trn[n].dtype.name=='category'):
            df[n] = c.astype('category').cat.as_ordered()
            df[n].cat.set_categories(trn[n].cat.categories, ordered=True, inplace=True)

def numericalize(df, col, name, max_n_cat):
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)      


# End fast.ai funcitons


# This reduce memory function came from: https://www.kaggle.com/siavrez

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    #return df

In [5]:
df_cal = pd.read_csv('calendar.csv', parse_dates= ['date'])
df_sales_train = pd.read_csv('sales_train_validation.csv')
df_prices = pd.read_csv('sell_prices.csv')
df_submissions = pd.read_csv('sample_submission.csv')


In [6]:
df_cal.shape

(1969, 14)

In [7]:
df_sales_train.shape, df_prices.shape, df_submissions.shape

((30490, 1919), (6841121, 4), (60980, 29))

In [8]:
#sales_train data provided
df_sales_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [9]:
#submission format
df_submissions.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## We need to adjust the format of the data given to work with prediction models

In [10]:
# All columns which are not days we will put into a list

list_id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] 
df_d_cols = df_sales_train.drop(list_id_vars, axis=1)
#date columns assigned to a new variable 
df_d_cols.columns

Index(['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1913)

In [11]:
# Pandas melt function UNPIVOTS column values and puts them into rows. Row for every day for an item/store combination. 
# This puts data into a flat format that will allow us to merge the sales train data with valendar and pricing data sets
%time df_melted_sales = df_sales_train.melt(id_vars = list_id_vars, value_vars = df_d_cols.columns, var_name = 'd', value_name = 'sales') 

Wall time: 14.4 s


In [12]:
# resulting melted sales data
df_melted_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [13]:
# These columns are redundant to ID - drop them.
# This notebook keeps running into the 16 GB of RAM given in free Kaggle.
# trying to keep the DFs as small as possible.
df_melted_sales.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1, inplace = True) 
df_melted_sales.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1,0
1,HOBBIES_1_002_CA_1_validation,d_1,0
2,HOBBIES_1_003_CA_1_validation,d_1,0
3,HOBBIES_1_004_CA_1_validation,d_1,0
4,HOBBIES_1_005_CA_1_validation,d_1,0


In [14]:
sub_cols = df_submissions.drop(['id'], axis=1).columns

# Like what we did to the sales data, we also need to melt the submission 
df_melted_sub = df_submissions.melt(id_vars = ['id'], value_vars = sub_cols, 
                                    var_name = 'd', value_name = 'sales') 
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,F1,0
1,HOBBIES_1_002_CA_1_validation,F1,0
2,HOBBIES_1_003_CA_1_validation,F1,0
3,HOBBIES_1_004_CA_1_validation,F1,0
4,HOBBIES_1_005_CA_1_validation,F1,0


In [15]:
df_melted_sub['d'] = df_melted_sub['d'].str.replace('F','')
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,1,0
1,HOBBIES_1_002_CA_1_validation,1,0
2,HOBBIES_1_003_CA_1_validation,1,0
3,HOBBIES_1_004_CA_1_validation,1,0
4,HOBBIES_1_005_CA_1_validation,1,0


In [16]:
df_melted_sub['d'] = pd.to_numeric(df_melted_sub['d'], errors='coerce') 
df_melted_sub['d'] = df_melted_sub['d'] + 1913
df_melted_sub = df_melted_sub.applymap(str)
df_melted_sub['d'] = 'd_'+ df_melted_sub['d'].astype(str)
df_melted_sub.head()

,id,d,sales
0,HOBBIES_1_001_CA_1_validation,d_1914,0
1,HOBBIES_1_002_CA_1_validation,d_1914,0
2,HOBBIES_1_003_CA_1_validation,d_1914,0
3,HOBBIES_1_004_CA_1_validation,d_1914,0
4,HOBBIES_1_005_CA_1_validation,d_1914,0


In [17]:
# As per the contest notes, this should end at "d_1941," so that's what we should see here...
df_melted_sub.tail() 

,id,d,sales
1707435,FOODS_3_823_WI_3_evaluation,d_1941,0
1707436,FOODS_3_824_WI_3_evaluation,d_1941,0
1707437,FOODS_3_825_WI_3_evaluation,d_1941,0
1707438,FOODS_3_826_WI_3_evaluation,d_1941,0
1707439,FOODS_3_827_WI_3_evaluation,d_1941,0


## Free up some RAM¶
The free Kaggle notebook only gives 16GB of RAM. So we'll delete the initial DFs that were merged into larger DFs and run the "reduce_mem" function on all DFs that we are going to be using in the next section of the notebook.

In [18]:
%who DataFrame

df_cal	 df_d_cols	 df_melted_sales	 df_melted_sub	 df_prices	 df_sales_train	 df_submissions	 


In [19]:
del df_sales_train
del df_submissions
del df_d_cols
gc.collect()

0

In [20]:
reduce_mem_usage(df_cal)
reduce_mem_usage(df_melted_sales)
reduce_mem_usage(df_melted_sub) 
reduce_mem_usage(df_prices)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 1001.26 Mb (25.0% reduction)
Mem. usage decreased to 39.08 Mb (0.0% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


## Split 
I'm using a smaller subet of data because it'll be faster to read and predict and to be able to keep it all in the free Kaggle RAM. Note that because this is a time series problem I'm not using scikit-learn train test split. This is because we want to train with older data and test with new data to simulate actual conditions. You will have all historical data, and will try to predict the next few months... Train test split will randomly split the data.

In [21]:
# limited the sample size so we can keep it in RAM in a free Kaggle kernel.
# Doing it as an 80/20 split as is standard for train/test.
df_test = df_melted_sales.tail(20000) 
df_train = df_melted_sales.iloc[-120000:-20000] 
df_submission = df_melted_sub.copy() # Copied just to keep naming conventions consistent

In [22]:
df_train.tail()

,id,d,sales
58307365,HOUSEHOLD_2_243_CA_4_validation,d_1913,0
58307366,HOUSEHOLD_2_244_CA_4_validation,d_1913,0
58307367,HOUSEHOLD_2_245_CA_4_validation,d_1913,0
58307368,HOUSEHOLD_2_246_CA_4_validation,d_1913,0
58307369,HOUSEHOLD_2_247_CA_4_validation,d_1913,3


In [23]:
# If this was done right, the number in the id and d columns below should pick up where it left off above
df_test.head() 

,id,d,sales
58307370,HOUSEHOLD_2_248_CA_4_validation,d_1913,2
58307371,HOUSEHOLD_2_249_CA_4_validation,d_1913,0
58307372,HOUSEHOLD_2_250_CA_4_validation,d_1913,0
58307373,HOUSEHOLD_2_251_CA_4_validation,d_1913,0
58307374,HOUSEHOLD_2_252_CA_4_validation,d_1913,0


In [24]:
df_train.shape, df_test.shape, df_submission.shape

((100000, 3), (20000, 3), (1707440, 3))

## Merge

Now we'll combine all the reference data to the training and test data into the training and test data frames.

In addition to the standard train/test dfs, we're also going to take the same steps with the submission data.

In [25]:
# Merge the calendar data with the training, test and submission DFs
df_train = df_train.merge(df_cal, left_on='d', right_on='d', how='left')
df_test = df_test.merge(df_cal, left_on='d', right_on='d', how='left')
df_submission = df_submission.merge(df_cal, left_on='d', right_on='d', how='left')
df_train.shape, df_test.shape, df_submission.shape # should be same numb of cols

((100000, 16), (20000, 16), (1707440, 16))

In [26]:
# Because we dropped columns from the training/test/submit DFs, we have to do this to be able to join to prices.
df_prices['id'] = df_prices['item_id'] +'_' + df_prices['store_id']
df_prices.drop(['item_id', 'store_id'], axis=1, inplace = True)
df_prices.head()

,wm_yr_wk,sell_price,id
0,11325,9.578125,HOBBIES_1_001_CA_1
1,11326,9.578125,HOBBIES_1_001_CA_1
2,11327,8.257812,HOBBIES_1_001_CA_1
3,11328,8.257812,HOBBIES_1_001_CA_1
4,11329,8.257812,HOBBIES_1_001_CA_1


In [28]:
# The training and submission data have a suffix that's not in the price data.
# So here we're creating a column that matches the price id column.
df_train['id_for_price'] = df_train['id'].str.replace('_validation','')
df_test['id_for_price'] = df_test['id'].str.replace('_validation','')
df_submission['id_for_price'] = df_submission['id'].str.replace('_evaluation','')
df_submission['id_for_price'] = df_submission['id_for_price'].str.replace('_validation','')

In [29]:
# Merge the price data with the training, test and submission DFs
df_train = pd.merge(df_train, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                    right_on = ['id', 'wm_yr_wk'])
df_test = pd.merge(df_test, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                   right_on = ['id', 'wm_yr_wk'])
df_submission = pd.merge(df_submission, df_prices,  how='left', left_on=['id_for_price', 'wm_yr_wk'], 
                         right_on = ['id', 'wm_yr_wk'])

In [30]:
# Check to see if there are any missing prices
df_train['sell_price'].count(), df_train.shape, df_test['sell_price'].count(), df_test.shape, df_submission['sell_price'].count(), df_submission.shape

(100000, (100000, 19), 20000, (20000, 19), 1707440, (1707440, 19))

In [31]:
# Merging with price added a bunch of unneeded columns
df_train.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_train.rename(columns = {"id_x":"id"}, inplace = True)
df_test.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_test.rename(columns = {"id_x":"id"}, inplace = True)
df_submission.drop(['id_for_price', 'id_y'], axis=1, inplace = True) 
df_submission.rename(columns = {"id_x":"id"}, inplace = True)

In [32]:
%who DataFrame


df_cal	 df_melted_sales	 df_melted_sub	 df_prices	 df_submission	 df_test	 df_train	 


In [33]:
# To save some RAM we will get rid of the dataframes that we merged from and are not in use now.
del df_cal
del df_melted_sales
del df_melted_sub
del df_prices
gc.collect()

209

In [34]:
reduce_mem_usage(df_train)
reduce_mem_usage(df_test)
reduce_mem_usage(df_submission)

Mem. usage decreased to  8.11 Mb (0.0% reduction)
Mem. usage decreased to  1.60 Mb (1.2% reduction)
Mem. usage decreased to 148.18 Mb (0.0% reduction)


In [35]:
# Saving the DFs for easy rollback and as a good starting point when reopening the notebook.

# Pickles
df_train.to_pickle('df_train_pickle.pkl')
df_test.to_pickle('df_test_pickle.pkl')
df_submission.to_pickle('df_submissions_pickle.pkl')

In [36]:
# Uncomment to load the pickle files...
df_train = pd.read_pickle('df_train_pickle.pkl')
df_test = pd.read_pickle('df_test_pickle.pkl')
df_submission = pd.read_pickle('df_submissions_pickle.pkl')

## Transform to get all the data in the DFs numeric and remove nulls


In [37]:
#cols_with_missing = df_merged.columns[df_merged.isnull().any()].tolist()
df_train.columns[df_train.isnull().any()]

Index(['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], dtype='object')

In [38]:
# Before removing nulls add a boolean column to declare if the value was missing
cols_with_missing = df_train.columns[df_train.isnull().any()].tolist()

for col in cols_with_missing:
    df_train[col + '_was_missing'] = df_train[col].isnull()
    df_test[col + '_was_missing'] = df_test[col].isnull()
    df_submission[col + '_was_missing'] = df_submission[col].isnull()

In [39]:
imputer_values = {'event_name_1' :'None', 'event_type_1' :'None', 
                  'event_name_2' :'None', 'event_type_2' :'None'                  
                 }
df_train.fillna(value = imputer_values, inplace = True)
df_test.fillna(value = imputer_values, inplace = True)
df_submission.fillna(value = imputer_values, inplace = True)

# Checking to see if there are any missing values
# The number of rows in each DF should be equal to the count
df_train.shape, df_train.count(), df_test.shape, df_test.count(), df_submission.shape, df_submission.count()

((100000, 21),
 id                          100000
 d                           100000
 sales                       100000
 date                        100000
 wm_yr_wk                    100000
 weekday                     100000
 wday                        100000
 month                       100000
 year                        100000
 event_name_1                100000
 event_type_1                100000
 event_name_2                100000
 event_type_2                100000
 snap_CA                     100000
 snap_TX                     100000
 snap_WI                     100000
 sell_price                  100000
 event_name_1_was_missing    100000
 event_type_1_was_missing    100000
 event_name_2_was_missing    100000
 event_type_2_was_missing    100000
 dtype: int64,
 (20000, 21),
 id                          20000
 d                           20000
 sales                       20000
 date                        20000
 wm_yr_wk                    20000
 weekday                 

In [40]:
#list of all non-numeric data
df_train.select_dtypes(include = 'object').columns 

Index(['id', 'd', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2'],
      dtype='object')

In [41]:
#we're going to use pandas date exploding features, so the given dates will wind up being redundant
drop_fields = ['weekday', 'year', 'wday', 'month']
df_train.drop(drop_fields, axis = 1, inplace = True)
df_test.drop(drop_fields, axis = 1, inplace = True)
df_submission.drop(drop_fields, axis = 1, inplace = True)
df_train.shape, df_test.shape, df_submission.shape

((100000, 17), (20000, 17), (1707440, 17))

In [42]:
# The next cell will change the IDs into categorical values.
# But the submission file needs them in the original format, so saving them to a series for use below
df_sub_ids = df_submission['id'].copy()

# We'll need the d column for the submission file format.
# But we can drop it from the train/test dfs as we already have date information in the DF
df_train.drop(['d'], axis = 1, inplace = True)
df_test.drop(['d'], axis = 1, inplace = True)
df_sub_d = df_submission.pop('d')

In [43]:
# Label encode
# For RAM considerations, we're not going to one hot encode anything.
# These are functions from fast.ai that convert objects to categories and keeps those categories consistent across training and test
train_cats(df_train)
apply_cats(df_test, df_train)
apply_cats(df_submission, df_train)
df_train.shape, df_test.shape, df_submission.shape

# Change categories to numbers
cat_cols = df_train.select_dtypes(include = 'category').columns
for i in cat_cols:
    df_train['cat_'+i] = df_train[i].cat.codes
    df_test['cat_'+i] = df_test[i].cat.codes
    df_submission['cat_'+i] = df_submission[i].cat.codes

df_train.drop(cat_cols, axis = 1, inplace = True)
df_test.drop(cat_cols, axis = 1, inplace = True)
df_submission.drop(cat_cols, axis = 1, inplace = True)

# Should return an empty list
df_train.select_dtypes(include = 'category').columns 

Index([], dtype='object')

In [44]:
# Should return an empty list
df_train.select_dtypes(include = 'object').columns

Index([], dtype='object')

In [45]:
# Explode out the date column into a ton of columns that a model can use
date_cols = df_train.select_dtypes(include = 'datetime64').columns
for i in date_cols:
    add_datepart(df_train, i)
    add_datepart(df_test, i)
    add_datepart(df_submission, i)

# Should see many more columns at the end of the list (year, month... is_year_end, etc) 
# Should also not see a "date" column
df_train.columns 

Index(['sales', 'wm_yr_wk', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price',
       'event_name_1_was_missing', 'event_type_1_was_missing',
       'event_name_2_was_missing', 'event_type_2_was_missing', 'cat_id',
       'cat_event_name_1', 'cat_event_type_1', 'cat_event_name_2',
       'cat_event_type_2', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [46]:
# should only see number and boolean columns
df_train.dtypes 

sales                         int16
wm_yr_wk                      int16
snap_CA                        int8
snap_TX                        int8
snap_WI                        int8
sell_price                  float16
event_name_1_was_missing       bool
event_type_1_was_missing       bool
event_name_2_was_missing       bool
event_type_2_was_missing       bool
cat_id                        int16
cat_event_name_1               int8
cat_event_type_1               int8
cat_event_name_2               int8
cat_event_type_2               int8
Year                          int64
Month                         int64
Week                          int64
Day                           int64
Dayofweek                     int64
Dayofyear                     int64
Is_month_end                   bool
Is_month_start                 bool
Is_quarter_end                 bool
Is_quarter_start               bool
Is_year_end                    bool
Is_year_start                  bool
Elapsed                     

In [47]:
reduce_mem_usage(df_train)
reduce_mem_usage(df_test)
reduce_mem_usage(df_submission)

Mem. usage decreased to  4.20 Mb (50.6% reduction)
Mem. usage decreased to  0.82 Mb (51.1% reduction)
Mem. usage decreased to 83.05 Mb (46.3% reduction)


In [48]:
gc.collect()

20

# Train and Predict 

In [49]:
# train and test we'll use below, the submission file sales column is all 0's so, we can just get rid of it.
y_train = df_train.pop('sales')
y_test = df_test.pop('sales')
df_submission.drop(['sales'], axis =1, inplace = True)

In [50]:
#Random Forest Model

m = RandomForestRegressor(n_jobs =-1) 
%time m.fit(df_train, y_train)


Wall time: 3.62 s


RandomForestRegressor(n_jobs=-1)

In [51]:
#R squared scoring - 
m.score (df_test, y_test) 

0.272496518935363

In [52]:
# For making predictions that we'll submit put train and test together and fit the model again for better predictions
train_test_concat = pd.concat([df_test, df_train])
y_concat = pd.concat([y_test, y_train])
m.fit(train_test_concat, y_concat)

# new score
m.score (train_test_concat, y_concat)


0.9467190954428273

In [53]:
df_submission.dtypes

wm_yr_wk                      int16
snap_CA                        int8
snap_TX                        int8
snap_WI                        int8
sell_price                  float16
event_name_1_was_missing       bool
event_type_1_was_missing       bool
event_name_2_was_missing       bool
event_type_2_was_missing       bool
cat_id                        int16
cat_event_name_1               int8
cat_event_type_1               int8
cat_event_name_2               int8
cat_event_type_2               int8
Year                          int16
Month                          int8
Week                           int8
Day                            int8
Dayofweek                      int8
Dayofyear                     int16
Is_month_end                   bool
Is_month_start                 bool
Is_quarter_end                 bool
Is_quarter_start               bool
Is_year_end                    bool
Is_year_start                  bool
Elapsed                       int32
dtype: object

In [54]:
# And finally, let's make some predictions that we'll submit
predictions = m.predict(df_submission)
predictions

array([0.92, 0.09, 0.74, ..., 1.55, 0.7 , 1.72])

## Create submission file

Submission file must be in format required by Kaggle

In [55]:
# Add the predictions made above and the original ids back onto the dataframe
df_submission['sales'] = predictions
df_submission['id'] = df_sub_ids
df_submission['F'] = df_sub_d

In [56]:
d = {'id': df_sub_ids,'F': df_sub_d, 'preds': predictions}
sub_file = pd.DataFrame(data=d)
sub_file.head()

,id,F,preds
0,HOBBIES_1_001_CA_1_validation,d_1914,0.92
1,HOBBIES_1_002_CA_1_validation,d_1914,0.09
2,HOBBIES_1_003_CA_1_validation,d_1914,0.74
3,HOBBIES_1_004_CA_1_validation,d_1914,2.97
4,HOBBIES_1_005_CA_1_validation,d_1914,2.85


In [57]:
# As per the competition requirements, we have to change the "d" values to F values
sub_file['F'] = sub_file['F'].str.replace('d_','')
sub_file['F'] = pd.to_numeric(sub_file['F'], errors='coerce') 
sub_file['F'] = sub_file['F'] - 1913
sub_file['F'] = 'F'+ sub_file['F'].astype(str)
sub_file.head()

,id,F,preds
0,HOBBIES_1_001_CA_1_validation,F1,0.92
1,HOBBIES_1_002_CA_1_validation,F1,0.09
2,HOBBIES_1_003_CA_1_validation,F1,0.74
3,HOBBIES_1_004_CA_1_validation,F1,2.97
4,HOBBIES_1_005_CA_1_validation,F1,2.85


In [58]:
# Pivot and reset the index to flatten the file
sub_file = sub_file.pivot(index='id', columns='F', values='preds')
sub_file = sub_file.reset_index()
sub_file.head()

F,id,F1,F10,F11,F12,F13,F14,F15,F16,F17,...,F26,F27,F28,F3,F4,F5,F6,F7,F8,F9
0,FOODS_1_001_CA_1_evaluation,1.28,1.32,1.32,1.18,1.25,1.26,1.32,1.32,1.32,...,1.18,1.25,1.02,1.28,1.28,1.19,1.26,1.26,1.32,1.32
1,FOODS_1_001_CA_1_validation,1.28,1.32,1.32,1.18,1.25,1.26,1.32,1.32,1.32,...,1.18,1.25,1.02,1.28,1.28,1.19,1.26,1.26,1.32,1.32
2,FOODS_1_001_CA_2_evaluation,1.28,1.32,1.32,1.18,1.25,1.26,1.32,1.32,1.32,...,1.18,1.25,1.02,1.28,1.28,1.19,1.26,1.26,1.32,1.32
3,FOODS_1_001_CA_2_validation,2.15,1.69,1.69,1.50,1.66,1.94,1.69,1.69,1.69,...,1.50,1.66,1.67,2.15,2.15,2.07,2.31,1.94,1.69,1.69
4,FOODS_1_001_CA_3_evaluation,1.28,1.32,1.32,1.18,1.25,1.26,1.32,1.32,1.32,...,1.18,1.25,1.02,1.28,1.28,1.19,1.26,1.26,1.32,1.32


In [59]:
# And finally... we have our submisison file...
csv_submit = sub_file.to_csv('submission.csv', index = False)